In [1]:
# télécharger les data à l'adresse : https://grouplens.org/datasets/movielens/
# puis cliquer sur le lien "ml-100k.zip" de "older datasets"

In [2]:
from pyspark import SparkContext,SparkConf
conf = SparkConf().setMaster("local").setAppName("Movies")
sc = SparkContext(conf = conf)

In [3]:
# u.data contient les ratings :  user id | item id | rating | timestamp
lines = sc.textFile("/home/jovyan/work/MoviesDataFrame/ml-100k/u.data")
lines.take(10)

['0\t50\t5\t881250949',
 '0\t172\t5\t881250949',
 '0\t133\t1\t881250949',
 '196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488']

In [4]:
ratings = lines.map(lambda x: x.split()[2])
ratings.take(10)

['5', '5', '1', '3', '3', '1', '2', '1', '4', '2']

In [ ]:
result = ratings.countByValue()

In [ ]:
result

In [ ]:
films = lines.map(lambda x: x.split()[1])

In [ ]:
films.take(10)

In [ ]:
films = lines.map(lambda x: (int(x.split()[1]), 1))
films.take(10)

In [ ]:
filmNumbers = films.reduceByKey(lambda x, y: x + y)

In [ ]:
filmNumbers.take(12)
# ID Movie, counts 

In [ ]:
#reverse_filmNumbers = filmNumbers.map(lambda (y, x): (x, y))
#reverse_filmNumbers = filmNumbers.map(x => (x[1],x[0]))
#reverse_filmNumbers = filmNumbers.map(_ => (_.0,_.1))
reverse_filmNumbers = filmNumbers.map(lambda x: (x[1],x[0]))

In [ ]:
reverse_filmNumbers.take(12)

In [ ]:
sorted_movies = reverse_filmNumbers.sortByKey()
# utilisation de la méthod "collect()" car sorted_movies est un RDD et un RDD n'est pas un iterable 
results = sorted_movies.collect()
for result in results:
    print(result)
#counts, Id Movie    
    

In [ ]:
# Le film le plus populaire a comme ID : 50

In [ ]:
# filmNumbers: (Id,counts)
results2 = filmNumbers.sortBy(lambda x:x[1],False).collect()
for result in results2:
    print(result)

In [ ]:
# Exercice
tmp = [('a', 1), ('b', 2), ('1', 3), ('d', 4), ('2', 5)]
test1 = sc.parallelize(tmp)
sorted = test1.sortByKey()
sorted.take(10)

In [ ]:
def loadMovieNames():
    movieNames = {}
    with open("/home/jovyan/work/MoviesDataFrame/ml-100k/u.item",encoding = "ISO-8859-1") as f:
        for line in f:
            fields = line.split('|')
# movieNames[50] = "StarWars"            
            movieNames[int(fields[0])] = fields[1]
    return movieNames

In [ ]:
#Broadcast Variables : partager les variables entre les workers
nameDict = sc.broadcast(loadMovieNames())
#nameDict = loadMovieNames()
#astuce : pour accéder au contenu de la variable broadcast: nameDict.value[id]
#sortedMovies: (counts,movieID)
sortedMoviesWithNames = sorted_movies.map(lambda x : (nameDict.value[x[1]], x[0]))
results = sortedMoviesWithNames.collect()

for result in results:
    print(result)